***
# 1. Data Preparation
Review the puzzle data for the 'cube' puzzle type. 
I started with a look at the puzzle_info.csv file.
***

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santa-2023/sample_submission.csv
/kaggle/input/santa-2023/puzzles.csv
/kaggle/input/santa-2023/puzzle_info.csv


In [2]:
puzzle_info = pd.read_csv('/kaggle/input/santa-2023/puzzle_info.csv')
print(puzzle_info)

# Inspect the allowed moves for the first puzzle type, cube_2/2/2
print(puzzle_info['allowed_moves'][0])

       puzzle_type                                      allowed_moves
0       cube_2/2/2  {'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11,...
1       cube_3/3/3  {'f0': [0, 1, 2, 3, 4, 5, 44, 41, 38, 15, 12, ...
2       cube_4/4/4  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
3       cube_5/5/5  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
4       cube_6/6/6  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
5       cube_7/7/7  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
6       cube_8/8/8  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
7       cube_9/9/9  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
8    cube_10/10/10  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
9    cube_19/19/19  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
10   cube_33/33/33  {'f0': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, ...
11      wreath_6/6  {'l': [1, 2, 3, 4, 5, 0, 6, 7, 8, 9], 'r': [6,...
12      wreath_7/7  {'l': [1, 2, 3, 4, 5, 6, 0, 7, 8, 9, 10, 11], ...
13    wreath_12/12  

***
There are 3 puzzle types, each of various dimensions.  
The simplest is cube_2/2/2. There are 12 allowed moves: f0, f1, r0, r1, d0, d1, and their respective inverses.  
Each of the allowed moves is described by an array containing some arrangement of 24 integers: 0 to 23.  
  
Next, I inspected the puzzles.csv, and specifically the data relating to the first puzzle, cube_2/2/2.
***

In [3]:
puzzles = pd.read_csv('/kaggle/input/santa-2023/puzzles.csv')
print(puzzles)
print(puzzles[puzzles['puzzle_type'] == 'cube_2/2/2'])

# Print the initial and solution states just for the first cube_2/2/2 puzzle
initial_state_puzzle1 = puzzles.loc[puzzles['puzzle_type'] == 'cube_2/2/2', 'initial_state'].iloc[0]
solution_state_puzzle1 = puzzles.loc[puzzles['puzzle_type'] == 'cube_2/2/2', 'solution_state'].iloc[0]
print("Puzzle1 Initial State: \n", initial_state_puzzle1)
print("Puzzle1 Solution State: \n", solution_state_puzzle1)

      id puzzle_type                                     solution_state  \
0      0  cube_2/2/2    A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F   
1      1  cube_2/2/2    A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F   
2      2  cube_2/2/2    A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F   
3      3  cube_2/2/2    A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F   
4      4  cube_2/2/2    A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F   
..   ...         ...                                                ...   
393  393  globe_3/33  A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...   
394  394  globe_3/33  A;A;A;A;A;A;C;C;C;C;C;C;E;E;E;E;E;E;G;G;G;G;G;...   
395  395  globe_3/33  N0;N1;N2;N3;N4;N5;N6;N7;N8;N9;N10;N11;N12;N13;...   
396  396  globe_8/25  A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...   
397  397  globe_8/25  A;A;A;A;A;D;D;D;D;D;G;G;G;G;G;J;J;J;J;J;M;M;M;...   

                                         initial_state  num_wildcards  
0      D;E;D;A;E;B;A;B;C;A;

***
There are 30 examples of the cube_2/2/2 puzzle. 
The first of these puzzles is:

Initial State
D;E;D;A;E;B;A;B;C;A;C;A;D;C;D;F;F;F;E;E;B;F;B;C
Solution State
A;A;A;A;B;B;B;B;C;C;C;C;D;D;D;D;E;E;E;E;F;F;F;F

A 'move' describes the desired re-ordering of the array's contents.  
As the data has come from a CSV file, the datatypes are str, which need to be converted to numpy arrays for efficient manipulation.  
Here is code to perform such conversion and then a move using numpy.
***

In [4]:
# Convert the initial state from a string type into a numpy array
puzzle_info_puzzle1 = np.array(initial_state_puzzle1.split(';'))
print(puzzle_info_puzzle1)

# Convert the allowed moves from a string type into a numpy array
moves_puzzle1 = ast.literal_eval(puzzle_info['allowed_moves'][0])
move_f0 = np.array(moves_puzzle1['f0'])
print(move_f0)

# Apply the move to the initial state to generate a resultant state
next_state = puzzle_info_puzzle1[move_f0]
print(next_state)


['D' 'E' 'D' 'A' 'E' 'B' 'A' 'B' 'C' 'A' 'C' 'A' 'D' 'C' 'D' 'F' 'F' 'F'
 'E' 'E' 'B' 'F' 'B' 'C']
[ 0  1 19 17  6  4  7  5  2  9  3 11 12 13 14 15 16 20 18 21 10  8 22 23]
['D' 'E' 'E' 'F' 'A' 'E' 'B' 'B' 'D' 'A' 'A' 'A' 'D' 'C' 'D' 'F' 'F' 'B'
 'E' 'F' 'C' 'C' 'B' 'C']


***
The inverse of a move will essentially reverse it, so '-f0', when applied to next_state, will bring the array back to its initial state again.
***

In [5]:
# Create an array to store the inverse move
move_f0_inv = []

# Reverse move_f0 to create its inverse and store in move_f0_inv array
for i in range(len(puzzle_info_puzzle1)):
    index = np.where(move_f0 == i)[0]
    move_f0_inv.append(index[0].item())

# Apply the inverse of move f0 to return next_state to its previous state
reversed_state = next_state[move_f0_inv]
print(reversed_state)

# Test that the reversal has worked correctly
if np.array_equal(puzzle_info_puzzle1, reversed_state):
    print("Reverse successful")


['D' 'E' 'D' 'A' 'E' 'B' 'A' 'B' 'C' 'A' 'C' 'A' 'D' 'C' 'D' 'F' 'F' 'F'
 'E' 'E' 'B' 'F' 'B' 'C']
Reverse successful


***
Now I have a method to create all of the allowed moves and apply a move to a state.  
I'm going to create a class that will create instances of the cube puzzle, and a class method that will enable a set of allowed moves.
***

In [6]:
# Define a class for each puzzle type and a class method to enable the allowed moves

class AllowedMoves:
    # Class variable to store the moves table
    moves_table = {}
    
    # Load the allowed moves for a puzzle type into a table
    @classmethod
    def load_moves_table(cls, row):
        allowed_moves_dict = ast.literal_eval(row['allowed_moves'])
        
        # Create all the inverse moves and load into the table
        inverse_moves_dict = {}
        for key, moves in allowed_moves_dict.items():
            inverse_key = '-' + key
            inverse_moves = [moves.index(i) for i in range(len(moves))]
            inverse_moves_dict[inverse_key] = inverse_moves
            
        cls.moves_table[row['puzzle_type']] = {**allowed_moves_dict, **inverse_moves_dict}

    # Return the allowed moves for a particular puzzle type from the moves table
    @classmethod
    def get_moves(cls, puzzle_type):
        return cls.moves_table[puzzle_type]
    
    
class CubePuzzle:
    # Initialize an instance of a puzzle with its initial state, solution state, and allowed moves
    def __init__(self, puzzle_type, initial_state, solution_state):
        self.current_state = initial_state
        self.solution_state = solution_state
        self.allowed_moves = AllowedMoves.get_moves(puzzle_type)
    
    # Apply a specified move to the current state of the puzzle
    def move(self, move):
        if move in self.allowed_moves:
            self.current_state = self.current_state[np.array(self.allowed_moves[move])]
        else:
            print(f"Invalid move: {move}")
        
        return self.current_state
        
  

***
Now I can test loading the moves table, which will take place at the start of the program.
***

In [7]:
# Note, to reduce the time required to load the whole table, I have created a subset of rows to load instead
start_row = 1
end_row = 2
puzzle_info_cubes = puzzle_info.iloc[start_row - 1:end_row]

# Load puzzle_info into the allowed moves table one row at a time
for index, row in puzzle_info_cubes.iterrows():
    AllowedMoves.load_moves_table(row)
    
print(AllowedMoves.get_moves('cube_2/2/2'))


{'f0': [0, 1, 19, 17, 6, 4, 7, 5, 2, 9, 3, 11, 12, 13, 14, 15, 16, 20, 18, 21, 10, 8, 22, 23], 'f1': [18, 16, 2, 3, 4, 5, 6, 7, 8, 0, 10, 1, 13, 15, 12, 14, 22, 17, 23, 19, 20, 21, 11, 9], 'r0': [0, 5, 2, 7, 4, 21, 6, 23, 10, 8, 11, 9, 3, 13, 1, 15, 16, 17, 18, 19, 20, 14, 22, 12], 'r1': [4, 1, 6, 3, 20, 5, 22, 7, 8, 9, 10, 11, 12, 2, 14, 0, 17, 19, 16, 18, 15, 21, 13, 23], 'd0': [0, 1, 2, 3, 4, 5, 18, 19, 8, 9, 6, 7, 12, 13, 10, 11, 16, 17, 14, 15, 22, 20, 23, 21], 'd1': [1, 3, 0, 2, 16, 17, 6, 7, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13, 18, 19, 20, 21, 22, 23], '-f0': [0, 1, 8, 10, 5, 7, 4, 6, 21, 9, 20, 11, 12, 13, 14, 15, 16, 3, 18, 2, 17, 19, 22, 23], '-f1': [9, 11, 2, 3, 4, 5, 6, 7, 8, 23, 10, 22, 14, 12, 15, 13, 1, 17, 0, 19, 20, 21, 16, 18], '-r0': [0, 14, 2, 12, 4, 1, 6, 3, 9, 11, 8, 10, 23, 13, 21, 15, 16, 17, 18, 19, 20, 5, 22, 7], '-r1': [15, 1, 13, 3, 0, 5, 2, 7, 8, 9, 10, 11, 12, 22, 14, 20, 18, 16, 19, 17, 4, 21, 6, 23], '-d0': [0, 1, 2, 3, 4, 5, 10, 11, 8, 9, 14, 15, 12, 13,

In [8]:
# Main program control logic to solve each puzzle in turn - initially, I am only tackling the first 2 puzzles

# for i in range(len(puzzles)):   - This is the code that runs on ALL puzzles to use later
# The following code just runs on the first two puzzles

for i in range(2):
    # Extract the puzzle data from the puzzles dataframe, convert initial and solution states into numpy arrays
    puzzle_type = puzzles.iloc[i]['puzzle_type']
    initial_state = np.array(puzzles.iloc[i]['initial_state'].split(';'))
    solution_state = np.array(puzzles.iloc[i]['solution_state'].split(';'))
    
    # Extract puzzle_class and execute the related code
    puzzle_class, puzzle_dim = puzzle_type.split('_')
    match puzzle_class:
        case "cube":
            puzzle = CubePuzzle(puzzle_type, initial_state, solution_state)
            move = 'f0'
            current_state = puzzle.move(move)
            if np.array_equal(current_state, solution_state):
                print("SOLVED")
            else:
                print("NOT SOLVED")            
        case "wreath":
            print("wreath")
        case "globe":
            print("globe")
        case _:
            print(f"Puzzle id: {puzzle['id']} is not a valid puzzle type.")


NOT SOLVED
NOT SOLVED
